# Shubert Function

In [ ]:
import numpy as np
import plotly.graph_objects as go

In [ ]:
def shubert(x1, x2):
    sum1 = np.sum([i * np.cos((i + 1) * x1 + i) for i in range(1, 6)])
    sum2 = np.sum([i * np.cos((i + 1) * x2 + i) for i in range(1, 6)])
    return sum1 * sum2

In [ ]:
def build_coords(min_val, max_val):
    x = np.linspace(min_val, max_val, 400)
    y = np.linspace(min_val, max_val, 400)
    X, Y = np.meshgrid(x, y)
    shubert_vectorized = np.vectorize(shubert)
    Z = shubert_vectorized(X,Y)
    return X, Y, Z

In [ ]:
def plot_contour(X,Y,Z):
    fig = go.Figure(data=go.Contour(
        z=Z,
        x=X[0, :],
        y=Y[:, 0],
        colorscale='inferno',
        contours=dict(
            coloring='heatmap',
            showlabels=True,
            labelfont=dict(
                size=11,
                color='white',
            )
        )
    ))
    
    fig.update_layout(
        title='Mapa de contorno de la función Shubert',
        width=600,
        height=600
    )
    
    fig.show()


In [ ]:
def plot_function(X,Y,Z):
    fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='inferno')])
    fig.update_layout(title='Función Shubert', autosize=False,
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=65, t=90))

    fig.show()

In [ ]:
def plot_line(x,y,title,xaxis_title,yaxis_title):
    fig = go.Figure(
        data=[go.Scatter(x=x, y=y, mode="lines", line=dict(width=2, color='blue'))],
        layout=go.Layout(
            title_text=title,
            width=600,
            height=600,
            xaxis_title=xaxis_title,
            yaxis_title=yaxis_title,
            yaxis=dict(range=[0])
        )
    )
    
    fig.show()

In [ ]:
def make__animation(X,Y,Z,history):
    trajectory = [np.append(point, shubert(point[0], point[1])) for point in history]

    fig = go.Figure(data=[
        go.Surface(z=Z, x=X, y=Y, contours_z=dict(show=True, usecolormap=True, project_z=True), opacity=0.15, colorscale='plasma'),
    ]*2)

    fig.update_layout(title='Función Shubert', autosize=False,
                      width=500, height=500,
                      margin=dict(l=65, r=50, b=65, t=90))

    fig.frames = [
        go.Frame(
            data=[
                go.Scatter3d(
                    x=[coord[0]], y=[coord[1]], z=[coord[2]], mode='markers', marker=dict(size=8, color='green'),
                    opacity=1,
                    projection=dict(x=dict(show=True, opacity=0.7, scale=0.5), y=dict(show=True, opacity=0.7, scale=0.5), z=dict(show=True, opacity=0.7, scale=0.5)))
            ],
            name=str(k),
            traces=[1],
            layout=go.Layout(annotations=[
                dict(xref="paper", yref="paper", x=0.05, y=0.95,
                     text=f"Valor actual: {coord[2]}", showarrow=False, font=dict(size=14))
            ])
        )
        for k, coord in enumerate(trajectory)
    ]

    sliders = [{
        'steps': [{'args': [[f.name], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
                   'label': k, 'method': 'animate'} for k, f in enumerate(fig.frames)],
        'transition': {'duration': 300},
        'x': 0.1, 'y': 0, 'currentvalue': {'visible': True, 'prefix': 'Paso: '}
    }]

    fig.update_layout(
        height=800, width=800,
        updatemenus=[{
            'buttons': [
                {'args': [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}],
                 'label': 'Play',
                 'method': 'animate'},
                {'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                                   'mode': 'immediate',
                                   'transition': {'duration': 0}}],
                 'label': 'Pause',
                 'method': 'animate'}
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }],
        sliders=sliders
    )

    fig.show()
    #plot(fig, filename="recocido.html", auto_open=True)

## Gráfica en intervalo [-10,10]

In [ ]:
X, Y, Z = build_coords(-10,10)

In [ ]:
plot_function(X,Y,Z)

In [ ]:
plot_contour(X,Y,Z)

## Recocido simulado

In [ ]:
class SimulatedAnnealing:
    def __init__(self, max_iterations=1500, initial_temperature=1000, objective_function=shubert):
        self.max_iterations = max_iterations
        self.initial_temperature = initial_temperature
        self.cooling_factor = 0.95
        self.objective_function = objective_function
        
    def temperature_change(self, current_temperature, cooling_factor):
        return current_temperature * cooling_factor
            
        
    def simulated_annealing(self, initial, cooling_factor=0.95):
        current = initial
        current_value = self.objective_function(current[0], current[1])
        history = []
        values_history = []
        
        current_temperature = self.initial_temperature
        
        iterations = 0
        
        for _ in range(self.max_iterations):
            current_temperature = self.temperature_change(current_temperature, cooling_factor)
            
            successor = np.random.uniform(low=-10, high=10, size=2)
            successor_value = self.objective_function(successor[0], successor[1])
            
            delta_value = successor_value - current_value
            
            if delta_value < 0:
                current = successor
                current_value = successor_value
                
                history.append(current)
                values_history.append(current_value)
            else:
                acceptance_probability = np.exp(-delta_value / current_temperature)
                if np.random.random() < acceptance_probability:
                    current = successor
                    current_value = successor_value
                    
                    history.append(current)
                    values_history.append(current_value)
        
        return current, current_value, history, values_history

In [ ]:
cooling_factors = [0.94,0.945,0.95,0.995]
fig = go.Figure()

for cooling_factor in cooling_factors:
    current_temp = 1000
    x = []
    y = []
    for i in range(1500):
        x.append(i)
        y.append(current_temp)
        current_temp = SimulatedAnnealing().temperature_change(current_temp, cooling_factor)
    
    fig.add_trace(
        go.Scatter(x=x, y=y, mode="lines", line=dict(width=2), name=f"cooling_factor={cooling_factor}")
    )
    
    
fig.update_layout(height=600, width=800, title_text="Cambios de temperatura geometricos", xaxis_title="Iteraciones", yaxis_title="Temperatura")

fig.show()

In [ ]:
cooling_factors = [0.94,0.945,0.95,0.995]
fig = go.Figure()

for cooling_factor in cooling_factors:
    initial = np.random.uniform(low=-10, high=10, size=2)
    minimum, value, history, values_history = SimulatedAnnealing().simulated_annealing(initial,cooling_factor)
    x = list(range(len(values_history)))
    
    fig.add_trace(
        go.Scatter(x=x, y=values_history, mode="lines", line=dict(width=2), name=f"cooling_factor={cooling_factor}")
    )
    
fig.update_layout(height=600, width=800, title_text="Valores vs. No. de sucesores", xaxis_title="No. de sucesores", yaxis_title="Valores")

fig.show()

In [ ]:
initial = np.random.uniform(low=-10, high=10, size=2)
minimum, value, history, values_history = SimulatedAnnealing().simulated_annealing(initial,cooling_factor=0.945)

In [ ]:
make__animation(X,Y,Z,history)

## Algoritmo genético 

In [ ]:
class GeneticAlgorithm:
    def __init__(self, objective_function=shubert):
        self.objective_function = objective_function
        self.epsilon = 1
    
    def generate_random_individual(self):
        return np.random.uniform(low=-10, high=10, size=2)
        
    def compute_fitness(self, individual):
        return -self.objective_function(individual[0], individual[1])
    
    def roulette_selection(self, population):
        fitness_array = np.array([self.compute_fitness(individual) for individual in population])
        fitness_array = fitness_array + (np.abs(np.min(fitness_array)) + self.epsilon)
        sum_fitness = np.sum(fitness_array)
        selection_probability = fitness_array / sum_fitness
        selected_index = np.random.choice(range(len(population)), p=selection_probability)
        return population[selected_index]
    
    def tournament_selection(self, population):
        tournament = random.sample(population, 3)
        tournament.sort(key=lambda individual: self.compute_fitness(individual),reverse=True)
        return tournament[0]
    
    def crossover(self, parent1, parent2):
        beta = np.random.uniform(low=-0.25, high=1.25, size=parent1.shape[0])
        
        child = np.zeros(parent1.shape[0])
        
        for i in range(parent1.shape[0]):
            child[i] = (parent1[i] * beta[i]) + (parent2[i] * (1 - beta[i]))
    
        return child
    
    def mutation(self, individual):
        std = 20 / 6
        
        for i in range(len(individual)):
            individual[i] += np.random.normal(0, std)
            
            if individual[i] < -10:
                individual[i] = -10
            elif individual[i] > 10:
                individual[i] = 10
        
        return individual
    
    def genetic_algorithm(self, population_size=100, generations=100, crossover_probability=0.8, mutation_probability=0.02):
        population = [self.generate_random_individual() for _ in range(population_size)]
        history = []
        
        for generation in range(generations):
            
            population = sorted(population, key=lambda individual: self.compute_fitness(individual), reverse=True)
            fittest_individual = population[0]
            
            history.append(fittest_individual)
            
            new_population = []

            while len(new_population) < population_size:
                parent1 = self.tournament_selection(population)
                parent2 = self.tournament_selection(population)
                
                if np.random.random() < crossover_probability:
                    child = self.crossover(parent1, parent2)
                else:
                    child = random.choice([parent1, parent2])
                    
                if np.random.random() < mutation_probability:
                    child = self.mutation(child)
                
                new_population.append(child)
                
            population = new_population
        
        fittest_individual = population[0]
        
        return fittest_individual, history

In [ ]:
fittest_individual, history = GeneticAlgorithm().genetic_algorithm()

In [ ]:
make__animation(X,Y,Z,history)